<a href="https://colab.research.google.com/github/lonely-crab/article-parser/blob/main/ArticleParser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорты

In [ ]:
!pip uninstall firebase-admin google-genai mcp --yes
!pip install scholarly==1.7.11 httpx==0.24.1 beautifulsoup4 requests pandas tqdm lxml --quiet

import sys
import time
import logging
from typing import List, Optional, Dict
import requests
from bs4 import BeautifulSoup
import pandas as pd

from tqdm import tqdm
from fake_useragent import UserAgent
from scholarly import scholarly, ProxyGenerator, MaxTriesExceededException

import warnings
warnings.filterwarnings('ignore')

Found existing installation: firebase-admin 6.9.0
Uninstalling firebase-admin-6.9.0:
  Successfully uninstalled firebase-admin-6.9.0
Found existing installation: google-genai 1.51.0
Uninstalling google-genai-1.51.0:
  Successfully uninstalled google-genai-1.51.0
Found existing installation: mcp 1.21.2
Uninstalling mcp-1.21.2:
  Successfully uninstalled mcp-1.21.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 86.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
   

# Настройка логирования

In [ ]:
logging.basicConfig()

logger = logging.getLogger("ArticleParser")
logger.propagate = False
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter("\n[%(levelname)s] %(asctime)s : %(message)s", datefmt='%H:%M:%S')
handler.setFormatter(fmt=formatter)
logger.addHandler(hdlr=handler)

logger.info("Логирование успешно инициализировано")


[INFO] 16:29:12 : Логирование успешно инициализировано


# Настройка ключевых слов

In [ ]:
PRIMARY_KEYWORDS = ["Асинхронные микросервисы", "FastAPI", "архитектура", "best practices", "performance"]
SECONDARY_KEYWORDS = ["async/await patterns", "dependency injection", "connection pooling", "event loop", "ASGI"]
TERTIARY_KEYWORDS = ["asyncio", "PostgreSQL", "testing", "monitoring", "security", "WebSockets", "Docker", "Kubernetes"]

ALL_KEYWORDS = PRIMARY_KEYWORDS + SECONDARY_KEYWORDS + TERTIARY_KEYWORDS
logger.info(f"Ключевые слова загружены ({len(ALL_KEYWORDS)}): {ALL_KEYWORDS}")



[INFO] 16:29:12 : Ключевые слова загружены (18): ['Асинхронные микросервисы', 'FastAPI', 'архитектура', 'best practices', 'performance', 'async/await patterns', 'dependency injection', 'connection pooling', 'event loop', 'ASGI', 'asyncio', 'PostgreSQL', 'testing', 'monitoring', 'security', 'WebSockets', 'Docker', 'Kubernetes']


# Поиск на Habr.com

## Функция поиска на Habr

In [ ]:
def search_habr_articles(query: str, max_results: int = 10) -> List[Optional[Dict[str, str]]]:
    """Функция парсинга Habr.

    Args:
    ---
        query : str
          Ключевое слово для запроса.
        max_results : int
          Максимальное количество статей, возвращаемых при данном query.

    Returns:
    ---
      articles : List[Optional[Dict[str, str]]]
        Список (len <= max_results) статей, найденных по данному query.
    """

    # Обязательно добавляем в query слово Python, чтобы не получать нерелевантные статьи
    query = ' '.join(["Python", query])
    logger.info(f"Поиск статей Habr по запросу: '{query}'")
    BASE_URL = "https://habr.com/ru/search/"
    params = {"q": query, "target_type": "posts", "order": "relevance"}
    headers = {}

    try:
        res = requests.get(BASE_URL, params=params, headers=headers, timeout=15)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "html.parser")
        articles = []
        for article in soup.select("article"):
            title_tag = article.find("a", class_="tm-title__link")
            author_tag = article.find("a", class_="tm-user-info__username")
            if title_tag and author_tag:
                title = title_tag.text.strip()
                author = "https://habr.com" + author_tag["href"]
                url = "https://habr.com" + title_tag["href"]
                article_data = {
                    'query': query,
                    'title': title,
                    'url': url,
                    'author': author,
                    'source': 'Habr'
                }
                articles.append(article_data)
            if len(articles) >= max_results:
                break

        logger.info(f"Найдено статей на Habr: {len(articles)} по '{query}'")
        time.sleep(1)
        return articles
    except Exception as e:
        logger.error(f"Habr — ошибка при обработке '{query}': {e}")
        return []

## Процесс поиска на Habr

In [ ]:
habr_results = []
for kw in tqdm(ALL_KEYWORDS, desc="Поиск на Habr"):
    habr_results.extend(search_habr_articles(kw, max_results=3))

habr_df = pd.DataFrame(habr_results)
logger.info(f"Итого найдено {len(habr_df)} статей на Habr")


Поиск на Habr:   0%|          | 0/18 [00:00<?, ?it/s]


[INFO] 16:29:12 : Поиск статей Habr по запросу: 'Python Асинхронные микросервисы'

[INFO] 16:29:14 : Найдено статей на Habr: 3 по 'Python Асинхронные микросервисы'


Поиск на Habr:   6%|▌         | 1/18 [00:02<00:42,  2.52s/it]


[INFO] 16:29:15 : Поиск статей Habr по запросу: 'Python FastAPI'

[INFO] 16:29:16 : Найдено статей на Habr: 3 по 'Python FastAPI'


Поиск на Habr:  11%|█         | 2/18 [00:04<00:38,  2.43s/it]

...


[INFO] 16:29:47 : Поиск статей Habr по запросу: 'Python Docker'

[INFO] 16:29:49 : Найдено статей на Habr: 3 по 'Python Docker'


Поиск на Habr:  94%|█████████▍| 17/18 [00:37<00:02,  2.17s/it]


[INFO] 16:29:50 : Поиск статей Habr по запросу: 'Python Kubernetes'

[INFO] 16:29:51 : Найдено статей на Habr: 3 по 'Python Kubernetes'


Поиск на Habr: 100%|██████████| 18/18 [00:39<00:00,  2.19s/it]


[INFO] 16:29:52 : Итого найдено 54 статей на Habr


# Поиск на Google Scholar

## Настройка прокси для Google Scholar


In [ ]:
def setup_scholarly_proxy(api_key: Optional[str] = None) -> ProxyGenerator:
    """Настройка прокси для scholarly для избежания получения 429 ошибки (Too many requests).

    Args:
    ---
        api_key: str
          API ключ для ScraperAPI (есть бесплатный период, можно попробовать с ним).

    Returns:
    ---
        ProxyGenerator - генератор прокси.
    """

    pg = ProxyGenerator()
    success = False

    if api_key is not None:
        logger.info("Настройка прокси через ScraperAPI")
        while success == False:
            success = pg.ScraperAPI(str(api_key))
        if success:
            logger.info("Прокси через ScraperAPI успешно настроен")
        else:
            logger.info("Не удалось настроить ScraperAPI, используем бесплатные прокси")
            pg.FreeProxies()
    else:
        logger.info("Настройка бесплатных прокси")
        pg.FreeProxies()
        logger.info("Бесплатные прокси настроены")

    scholarly.use_proxy(pg)
    return pg

## Функция поиска на Google Scholar

In [ ]:
def search_google_scholar_with_proxy(query: str,
                                     max_results: int = 10,
                                     start_year=2020,
                                     pg: Optional[ProxyGenerator] = None) -> List[Optional[Dict[str, str]]]:
    """
    Поиск статей в Google Scholar с использованием прокси.

    Args:
    ---
      query : str
        Ключевое слово для запроса.
      max_results : int
        Максимальное количество статей, возвращаемых при данном query.
      start_year : int
        Год начала фильтрации.
      pg : ProxyGenerator
        Генератор прокси (если None, прокси не используется).

    Returns:
      articles : List[Optional[Dict[str, str]]]
        Список (len <= max_results) статей, найденных по данному query.
    """
    # Обязательно добавляем в query слово Python, чтобы не получать нерелевантные статьи
    query = ' '.join(["Python", query])

    logger.info(f"Поиск в Scholar по запросу: '{query}'")

    articles = []

    try:
        if pg: # Подключаем прокси
            scholarly.use_proxy(pg)


        # Выполняем поиск
        search_query = scholarly.search_pubs(query=query,
                                             year_low=start_year)

        count = 0
        for result in search_query:
            if count >= max_results:
                break

            try:
                bib = result.get('bib', {})

                title = bib.get('title', 'N/A')
                authors = bib.get('author', 'N/A')

                if isinstance(authors, list):
                    authors = ', '.join(authors)

                year = bib.get('pub_year', 'N/A')
                journal = bib.get('venue', bib.get('journal', 'N/A'))
                abstract = bib.get('abstract', 'N/A')

                # Извлекаем URL
                url = result.get('pub_url', result.get('eprint_url', 'N/A'))

                # Цитирования
                citations = result.get('num_citations', 0)

                article = {
                    'query': query,
                    'title': title,
                    'authors': authors,
                    'year': year,
                    'journal': journal,
                    'url': url,
                    'abstract': abstract if abstract != 'N/A' else 'N/A',
                    'citations': citations,
                    'source': 'Google Scholar'
                }

                articles.append(article)
                count += 1

                logger.info(f"[{count}/{max_results}] {title}...")

                # Важно: задержка между запросами для уменьшения вероятности отказа сервиса
                time.sleep(30)

            except StopIteration:
                logger.info(f"Достигнут конец результатов")
                break
            except MaxTriesExceededException as e:
                logger.error(f"MaxTriesExceededException ошибка при обработке результата: {e}")
                continue
            except Exception as e:
                logger.error(f"Другая ошибка при обработке результата: {e}")
                continue

        logger.info(f"Найдено {len(articles)} статей по запросу '{query}'")

    except Exception as e:
        logger.error(f"Ошибка при поиске '{query}': {e}")

    return articles

## Установка прокси и поиск статей

In [ ]:
API_KEY = None # рекомендую добавить api ключ если он есть, с ним гораздо надежнее
pg = setup_scholarly_proxy(api_key=API_KEY)

all_results = []

for keyword in tqdm(ALL_KEYWORDS, desc="Поиск на Google Scholar"):
  try:
    results = search_google_scholar_with_proxy(
        query=keyword,
        max_results=5,
        start_year=2023
    )
    all_results.extend(results)
  except Exception as e:
    logger.info(f"Запрос '{keyword}' завершился ошибкой и был пропущен: {e}")


    # Задержка между ключевыми словами
    pause = 30
    logger.info(f"\nПауза {pause} секунд перед следующим запросом")
    time.sleep(pause)

scholar_df = pd.DataFrame(all_results)

# Удаляем дубликаты
if not scholar_df.empty:
    scholar_df = scholar_df.drop_duplicates(subset=['title'], keep='first')
    scholar_df = scholar_df.sort_values('citations', ascending=False)

logger.info(f"\nИТОГО найдено {len(scholar_df)} уникальных статей")


[INFO] 16:29:52 : Настройка бесплатных прокси

[INFO] 16:29:53 : Бесплатные прокси настроены


Поиск на Google Scholar:   0%|          | 0/18 [00:00<?, ?it/s]


[INFO] 16:29:55 : Поиск в Scholar по запросу: 'Python Асинхронные микросервисы'

[ERROR] 16:31:02 : Ошибка при поиске 'Python Асинхронные микросервисы': Cannot Fetch from Google Scholar.


Поиск на Google Scholar:   6%|▌         | 1/18 [01:07<19:13, 67.83s/it]


[INFO] 16:31:02 : Поиск в Scholar по запросу: 'Python FastAPI'

[INFO] 16:31:09 : Найдено 0 статей по запросу 'Python FastAPI'


...


[INFO] 17:02:01 : Поиск в Scholar по запросу: 'Python Docker'

[INFO] 17:02:21 : Найдено 0 статей по запросу 'Python Docker'


Поиск на Google Scholar:  94%|█████████▍| 17/18 [32:25<02:08, 128.17s/it]


[INFO] 17:02:21 : Поиск в Scholar по запросу: 'Python Kubernetes'

[INFO] 17:02:31 : [1/5] Design of Application Development Infrastructure Automation Tools on Kubernetes Using Python...

[INFO] 17:03:01 : [2/5] CONTAINERIZING PYTHON APPLICATIONS USING DOCKER TO BUILD A MICROSERVICE ARCHITECTURE...

[INFO] 17:03:31 : [3/5] Building AI-driven cloud-native applications with Kubernetes and containerization...

[INFO] 17:04:01 : [4/5] Dynamic Load Balancing in Kubernetes Environments With Kubernetes Scheduling Extension (KSE)...

[INFO] 17:04:31 : [5/5] Análise de Auto-Scaling em Plataformas Serverless no Kubernetes...

[INFO] 17:05:01 : Найдено 5 статей по запросу 'Python Kubernetes'


Поиск на Google Scholar: 100%|██████████| 18/18 [35:06<00:00, 117.04s/it]


[INFO] 17:05:01 : 
ИТОГО найдено 49 уникальных статей


# Итог: Сбор всех результатов и сохранение

In [ ]:
with pd.ExcelWriter("literature_search_results.xlsx") as writer:
    habr_df.to_excel(writer, sheet_name="Habr", index=False)
    scholar_df.to_excel(writer, sheet_name="GoogleScholar", index=False)
    # elibrary_df.to_excel(writer, sheet_name="eLibrary", index=False)

habr_df.to_csv("habr_results.csv", index=False)
scholar_df.to_csv("scholar_results.csv", index=False)
# elibrary_df.to_csv("elibrary_results.csv", index=False)
logger.info("Результаты поиска статей успешно сохранены в Excel и CSV-файлы")



[INFO] 17:05:02 : Результаты поиска статей успешно сохранены в Excel и CSV-файлы
